# **TULU-3 Fine-Tuning**
In this hands-on exercise, we will be fine-tuning different models for various tasks using classical fine-tuning. Classical fine-tuning is a common approach to establish a solid baseline for model specialization performance.

The goal of fine-tuning is to take a pre-trained model and adapt it to a specific task or dataset. By leveraging the knowledge and representations learned from a large-scale pre-training task, we can achieve better performance on downstream tasks with less training data.

In [65]:
from pathlib import Path
import os
import datasets
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForCausalLM, AutoModelForSeq2SeqLM
from torch.utils.data import Dataset, DataLoader, IterableDataset
from datasets import load_dataset
import torch

from utils import sft_tulu_tokenize_and_truncate

from tqdm.notebook import trange, tqdm


DSDIR = Path(os.environ['DSDIR'])
os.environ['TOKENIZERS_PARALLELISM'] = 'false'


In [72]:
DSDIR = Path(os.environ["DSDIR"])
model_path = DSDIR / "HuggingFace_Models" / "Qwen/Qwen2.5-14B"
dataset_path = DSDIR / "HuggingFace" / "allenai" / "tulu-3-sft-mixture" / "data" /"*.parquet"
#dataset_path = "/lustre/fswork/dataset/tulu-3-sft-mixture/data/*.parquet"

---

In [5]:
dataset_path

PosixPath('/lustre/fsmisc/dataset/HuggingFace/allenai/tulu-3-sft-mixture/data/*.parquet')

In [6]:
dataset = load_dataset("parquet", data_files=str(dataset_path), split="train")
tokenizer = AutoTokenizer.from_pretrained(str(model_path) + '-Instruct', padding_side="left")
tokenizer.pad_token_id = tokenizer.eos_token_id

In [7]:
len(dataset)

939343

In [25]:
l = 0
for r, row in enumerate(dataset):
    messages = row["messages"]
    if len(messages) > l:
        l = len(messages)
        print(f"index {r} : len {l}")

index 0 : len 2
index 6 : len 4
index 178 : len 6
index 97359 : len 7
index 97398 : len 9
index 98190 : len 19
index 100177 : len 21
index 106859 : len 40
index 107026 : len 44
index 107050 : len 100
index 110345 : len 134
index 112803 : len 174
index 116270 : len 198
index 121510 : len 208
index 147532 : len 226
index 160167 : len 234
index 163810 : len 256
index 184526 : len 264
index 190694 : len 294


In [26]:
row = dataset[178]
row

{'id': 'oasst1_5779',
 'messages': [{'content': 'What are some types of breeds of medium sized dogs that might be good starter dogs for a young family?',
   'role': 'user'},
  {'content': "Here are some medium sized dog breeds that are suitable for first time dog owners:\n1. Golden retrievers: Golden retrievers are gentle and have a sense of loyalty which makes them reliable and trust-worthy. They are also easy to train and are energetic during outdoor play.\n2. Labradors: Labradors are a loyal companion and act like your best friend which is why they're the perfect family, they're also patient and gentle unless annoyed. They are also a perfect watchdog as they have a loud bark.\n3. Greyhounds: Greyhounds are intelligent, clean, quite and easy to live with which makes them the perfect dog for people that prefer to stay at home.\nUltimately, it's up to you to decide which breed you find the best, based on your needs and preferences. Good luck! 😊",
   'role': 'assistant'},
  {'content': 

In [73]:
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype="bfloat16")

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [74]:
len(model.model.layers)

48

In [75]:
model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 5120)
    (layers): ModuleList(
      (0-47): 48 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=5120, out_features=5120, bias=True)
          (k_proj): Linear(in_features=5120, out_features=1024, bias=True)
          (v_proj): Linear(in_features=5120, out_features=1024, bias=True)
          (o_proj): Linear(in_features=5120, out_features=5120, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=5120, out_features=13824, bias=False)
          (up_proj): Linear(in_features=5120, out_features=13824, bias=False)
          (down_proj): Linear(in_features=13824, out_features=5120, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((5120,), eps=1e-05)
        (post_attention_layernorm): Qwen2RMSNorm((5120,), eps=1e-05)
      )
    )
    (norm): Qwen2RMSNorm((5120,), eps=1e-05)
    (rotary_emb

----

Nombre de batchs pour 2 epochs

In [30]:
len(dataset)

939343

In [31]:
939343 // 128 * 2

14676

<hr style="border:1px solid red"> 


In [13]:
!mkdir slurm
!mkdir configs

mkdir: cannot create directory ‘slurm’: File exists


## FSDP2 + sAC

je mesure environ **10% d'augmentation** de throughput en utilisant l'AC dans le scenario 64GPU FSDP 14B entre un ratio 1/8 et 1. Sachant que la comm est une composante du temps importante.

Avec 32 GPU on a une augmentation du temps un peu en dessous de 2.

### H100

In [16]:
!sbatch slurm/FSDP_sAC_h100_14B.slurm

Submitted batch job 1701537


In [47]:
!sbatch slurm/FSDP_sAC_h100_32B.slurm

sbatch: IDRIS: setting exclusive mode for the job.
Submitted batch job 1691223


In [55]:
!sbatch slurm/FSDP_sAC_h100_72B.slurm

sbatch: IDRIS: setting exclusive mode for the job.
Submitted batch job 1691389


In [21]:
!squeue --me

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           1701231    gpu_p6       bc  ssos040 PD       0:00     16 (Priority)
           1701349    gpu_p6       bc  ssos040 PD       0:00     16 (Priority)
           1701537    gpu_p6       bc  ssos040 PD       0:00      1 (Priority)


### A100

In [6]:
!sbatch slurm/fsdp2_a100_14B.slurm

sbatch: IDRIS: setting exclusive mode for the job.
Submitted batch job 1662324


In [49]:
!sbatch slurm/fsdp2_a100_32B.slurm

sbatch: IDRIS: setting exclusive mode for the job.
Submitted batch job 1655334


In [50]:
!sbatch slurm/fsdp2_a100_72B.slurm

sbatch: IDRIS: setting exclusive mode for the job.
Submitted batch job 1655335


In [101]:
!squeue --me

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           1686022    gpu_p6       bc  ssos040  R       0:55     16 jzxh[088,148-149,174,190,258-262,319-322,346-347]


# TP + FSDP

## H100

In [106]:
!sbatch slurm/TP4_fsdp16_h100_14B.slurm

sbatch: IDRIS: setting exclusive mode for the job.
Submitted batch job 1688218


In [108]:
!squeue --me

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           1688218    gpu_p6       bc  ssos040  R       0:06     16 jzxh[088,148-149,174,190,258-262,319-322,346-347]


## Nemo

### Recette install 2

```bash

module load miniforge/
conda create --prefix /lustre/fsn1/projects/idris/sos/commun/CONDA_ENVS/NeMo/NeMO_2 python=3.12
conda activate /lustre/fsn1/projects/idris/sos/commun/CONDA_ENVS/NeMo/NeMO_2

module load arch/h100
module load gcc/11.3.1 cuda/12.8.0 cudnn/9.10.2.21-12-cuda nccl/2.27.3-1-cuda

pip install --upgrade pip
pip install --no-cache-dir torch==2.8.0 torchvision==0.23.0

module load libsndfile ffmpeg
pip install --no-cache-dir Cython packaging

#pip install --no-cache-dir nemo_toolkit['all']
pip install --no-cache-dir "git+https://github.com/NVIDIA/NeMo.git@main#egg=nemo_toolkit[all]"

pip install --no-cache-dir idr_torch
pip install --no-cache-dir --extra-index-url https://pypi.nvidia.com transformer_engine[pytorch]


```

### H100

In [15]:
%%writefile slurm/NeMO_h100_444.slurm
#!/bin/bash
#SBATCH --job-name=bc
#SBATCH --output=logs/out/NeMO_h100_444_%j.out 
#SBATCH --error=logs/err/NeMO_h100_444%j.err
#SBATCH --gres=gpu:4
#SBATCH --nodes=16
#SBATCH --ntasks-per-node=4
#SBATCH --hint=nomultithread 
#SBATCH --time=00:30:00
#SBATCH --cpus-per-task=24
#SBATCH -C h100
#SBATCH --partition=gpu_p6
#SBATCH --account=sos@h100

## load module 
module purge
module load arch/h100
module load nemo/2.4.0


## Distribution setup
export MASTER_ADDR=$(scontrol show hostnames $SLURM_JOB_NODELIST | head -n1)
export MASTER_PORT=29500
    

## launch script on every task 
set -x
time srun python -u nemo_megatron.py --model Qwen/Qwen2.5-14B-Instruct \
                                 --devices 4 \
                                 --num-nodes 16 \
                                 --dp-size 4 \
                                 --tp-size 4 \
                                 --pp-size 4 \
                                 --cp-size 1 \
                                 --accumulate-grad-batches 8
                                 #--virtual-pp-size \
                                 #--sequence-parallel \

date

Overwriting slurm/NeMO_h100_444.slurm


In [13]:
!sbatch slurm/NeMO_h100_444.slurm

sbatch: IDRIS: setting exclusive mode for the job.
Submitted batch job 1690342


In [14]:
!squeue --me

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           1690342    gpu_p6       bc  ssos040 PD       0:00     16 (Resources)
           1690297    gpu_p6       bc  ssos040  R       0:43     16 jzxh[088,148-149,174,190,258-262,319-322,346-347]


In [18]:
%%writefile slurm/NeMO_a100_444.slurm
#!/bin/bash
#SBATCH --job-name=bc
#SBATCH --output=logs/out/NeMO_a100_444_%j.out 
#SBATCH --error=logs/err/NeMO_a100_444%j.err
#SBATCH --gres=gpu:8
#SBATCH --nodes=8
#SBATCH --ntasks-per-node=8
#SBATCH --hint=nomultithread 
#SBATCH --time=03:00:00
#SBATCH --cpus-per-task=8
#SBATCH -C a100
#SBATCH --partition=gpu_p5
#SBATCH --account=sos@a100

## load module 
module purge
module load arch/a100
module load nemo/2.3.1


## Distribution setup
export MASTER_ADDR=$(scontrol show hostnames $SLURM_JOB_NODELIST | head -n1)
export MASTER_PORT=29500
    

## launch script on every task 
set -x
time srun python -u nemo_megatron.py --model Qwen/Qwen2.5-14B-Instruct \
                                 --devices 8 \
                                 --num-nodes 8 \
                                 --dp-size 4 \
                                 --tp-size 4 \
                                 --pp-size 4 \
                                 --cp-size 1 \
                                 --accumulate-grad-batches 8
                                 #--virtual-pp-size \
                                 #--sequence-parallel \
                                 
date

Overwriting slurm/NeMO_a100_444.slurm


In [19]:
!sbatch slurm/NeMO_a100_444.slurm

sbatch: IDRIS: setting exclusive mode for the job.
Submitted batch job 1651301


In [21]:
!squeue --me

             JOBID PARTITION     NAME     USER ST          START_TIME  NODES SCHEDNODES           NODELIST(REASON)
           1662324    gpu_p5       bc  ssos040 PD 2025-11-14T14:04:20      2 jean-zay-iam[39,52]  (Priority)
           1662313    gpu_p5       bc  ssos040 PD 2025-11-15T05:52:37      8 jean-zay-iam[01-03,0 (Priority)
           1663702    gpu_p6       bc  ssos040 PD                 N/A      8 (null)               (Resources)
           1663729    gpu_p5       bc  ssos040 PD                 N/A      2 (null)               (Priority)


In [21]:
!sinfo -s

PARTITION   AVAIL  TIMELIMIT   NODES(A/I/O/T) NODELIST
cpu_p1*        up 4-04:00:00     705/3/12/720 r1i0n[0-35],r1i1n[0-35],r1i2n[0-35],r1i3n[0-35],r1i4n[0-35],r1i5n[0-35],r1i6n[0-35],r1i7n[0-35],r2i0n[0-35],r2i1n[0-35],r2i2n[0-35],r2i3n[0-35],r2i4n[0-35],r2i5n[0-35],r2i6n[0-35],r2i7n[0-35],r3i0n[0-35],r3i1n[0-35],r3i2n[0-35],r3i3n[0-35]
gpu_p13        up 4-04:00:00     345/48/3/396 r3i4n[0-8],r3i5n[0-8],r3i6n[0-8],r3i7n[0-8],r6i0n[0-8],r6i1n[0-8],r6i2n[0-8],r6i3n[0-8],r6i4n[0-8],r6i5n[0-8],r6i6n[0-8],r6i7n[0-8],r7i0n[0-8],r7i1n[0-8],r7i2n[0-8],r7i3n[0-8],r7i4n[0-8],r7i5n[0-8],r7i6n[0-8],r7i7n[0-8],r8i0n[0-8],r8i1n[0-8],r8i2n[0-8],r8i3n[0-8],r8i4n[0-8],r8i5n[0-8],r8i6n[0-8],r8i7n[0-8],r9i0n[0-8],r9i1n[0-8],r9i2n[0-8],r9i3n[0-8],r9i4n[0-8],r9i5n[0-8],r9i6n[0-8],r9i7n[0-8],r10i0n[0-8],r10i1n[0-8],r10i2n[0-8],r10i3n[0-8],r10i4n[0-8],r10i5n[0-8],r10i6n[0-8],r10i7n[0-8]
gpu_v116    inact 4-04:00:00      126/0/0/126 r3i4n[0-8],r3i5n[0-8],r3i6n[0-8],r3i7n[0-8],r6i0n[0-8],r6i1n[0-8],r10i0n[0-

## NeMo - HFAutomodel

Note :
* problème avec le gradient cliping - Il faut le désactiver pour que ça marche !!
* desactiver le `sequence_parallel` - problème avec la loss

A tester:
* --use-chunked-ce
* --attn-implementation flash_attention_2
* --enable-grad-ckpt
* --cp-size

In [24]:
!sbatch slurm/HFAuto_h100_14B.slurm

sbatch: IDRIS: setting exclusive mode for the job.
Submitted batch job 1702383


In [26]:
!squeue --me --start

             JOBID PARTITION     NAME     USER ST          START_TIME  NODES SCHEDNODES           NODELIST(REASON)
           1701231    gpu_p6       bc  ssos040 PD                 N/A     16 (null)               (Priority)
           1701349    gpu_p6       bc  ssos040 PD                 N/A     16 (null)               (Priority)
           1701537    gpu_p6       bc  ssos040 PD                 N/A      1 (null)               (Priority)
           1702288    gpu_p6       bc  ssos040 PD                 N/A     16 (null)               (Priority)
           1702383    gpu_p6       bc  ssos040 PD                 N/A     16 (null)               (Resources)


In [6]:
!sinfo -s

PARTITION   AVAIL  TIMELIMIT   NODES(A/I/O/T) NODELIST
cpu_p1*        up 4-04:00:00     702/1/17/720 r1i0n[0-35],r1i1n[0-35],r1i2n[0-35],r1i3n[0-35],r1i4n[0-35],r1i5n[0-35],r1i6n[0-35],r1i7n[0-35],r2i0n[0-35],r2i1n[0-35],r2i2n[0-35],r2i3n[0-35],r2i4n[0-35],r2i5n[0-35],r2i6n[0-35],r2i7n[0-35],r3i0n[0-35],r3i1n[0-35],r3i2n[0-35],r3i3n[0-35]
gpu_p13        up 4-04:00:00      393/0/3/396 r3i4n[0-8],r3i5n[0-8],r3i6n[0-8],r3i7n[0-8],r6i0n[0-8],r6i1n[0-8],r6i2n[0-8],r6i3n[0-8],r6i4n[0-8],r6i5n[0-8],r6i6n[0-8],r6i7n[0-8],r7i0n[0-8],r7i1n[0-8],r7i2n[0-8],r7i3n[0-8],r7i4n[0-8],r7i5n[0-8],r7i6n[0-8],r7i7n[0-8],r8i0n[0-8],r8i1n[0-8],r8i2n[0-8],r8i3n[0-8],r8i4n[0-8],r8i5n[0-8],r8i6n[0-8],r8i7n[0-8],r9i0n[0-8],r9i1n[0-8],r9i2n[0-8],r9i3n[0-8],r9i4n[0-8],r9i5n[0-8],r9i6n[0-8],r9i7n[0-8],r10i0n[0-8],r10i1n[0-8],r10i2n[0-8],r10i3n[0-8],r10i4n[0-8],r10i5n[0-8],r10i6n[0-8],r10i7n[0-8]
gpu_v116    inact 4-04:00:00      126/0/0/126 r3i4n[0-8],r3i5n[0-8],r3i6n[0-8],r3i7n[0-8],r6i0n[0-8],r6i1n[0-8],r10i0n[0-

# Avec container

```
export SINGULARITY_BINDPATH=src1:dest1,src2:dest2,src3:dest3
export SINGULARITY_BINDPATH="/dev/infiniband,/etc/libibverbs.d,/etc/rdma,$WORK,$ALL_CCFRWORK,$DSDIR,$SCRATCH,$ALL_CCFRSCRATCH,$JOBSCRATCH"
```

```bash
srun singularity exec --nv \
  -B /dev/infiniband:/dev/infiniband \
  -B /sys/class/infiniband:/sys/class/infiniband \
  -B /sys/class/infiniband_verbs:/sys/class/infiniband_verbs \
  -B /sys/class/net:/sys/class/net \
  -B /etc/libibverbs.d:/etc/libibverbs.d \
  -B /etc/rdma:/etc/rdma \
  -B /usr/lib64/libfabric:/usr/lib64/libfabric \
  your_image.sif \
  bash -lc '
    export FI_PROVIDER=verbs
    export NCCL_IB_HCA=mlx5_0
    export NCCL_SOCKET_IFNAME=^lo,docker
    export UCX_TLS=rc,ud,cuda_copy,cuda_ipc,sm
    torchrun --nproc_per_node=8 train.py
  '
  ```

```bash
srun singularity exec \
  -B /dev/hfi1:/dev/hfi1 \
  -B /sys/class/infiniband:/sys/class/infiniband \
  -B /sys/class/infiniband_verbs:/sys/class/infiniband_verbs \
  -B /sys/class/net:/sys/class/net \
  -B /etc/libibverbs.d:/etc/libibverbs.d \
  -B /etc/rdma:/etc/rdma \
  -B /usr/lib64/libfabric:/usr/lib64/libfabric \
  -B /usr/lib64/libpsm2:/usr/lib64/libpsm2 \
  your_image.sif \
  bash -lc '
    export FI_PROVIDER=psm2
    export I_MPI_FABRICS=shm:ofi
    mpirun ./app
  '
```

```bash
# cleans out modules loaded in interactive and inherited by default
module purge
 
module load singularity
 
# echo des commandes lancées
set -x
 
time srun singularity exec --nv \
--bind .:$HOME,$JOBSCRATCH:$JOBSCRATCH \
$SINGULARITY_ALLOWED_DIR/MegaSingularity.sif \
python ./Megatron-LM/tasks/main.py --task IMDB --train-data ./imdb/dataset_train.csv --valid-data
```

In [29]:
%%writefile slurm/Sif_fsdp2_h100_14B.slurm
#!/bin/bash
#SBATCH --job-name=bc
#SBATCH --output=logs/out/Sif_fsdp2_h100_14B_%j.out 
#SBATCH --error=logs/err/Sif_fsdp2_h100_14B_%j.err
#SBATCH --gres=gpu:4
#SBATCH --nodes=16
#SBATCH --ntasks-per-node=4
#SBATCH --hint=nomultithread 
#SBATCH --time=00:30:00
#SBATCH --cpus-per-task=24
#SBATCH -C h100
#SBATCH --partition=gpu_p6
#SBATCH --account=sos@h100
##SBATCH --qos=qos_gpu_h100-dev 

## load module 
module purge
module load singularity

export NCCL_DEBUG=WARN

## Distribution setup
export MASTER_ADDR=$(scontrol show hostnames $SLURM_JOB_NODELIST | head -n1)
export MASTER_PORT=29500

export SINGULARITY_BINDPATH="/dev/infiniband,/etc/libibverbs.d,/etc/rdma,/sys/class/infiniband,/sys/class/infiniband_verbs,/sys/class/net,/run/udev,$WORK,$ALL_CCFRWORK,$DSDIR,$SCRATCH,$ALL_CCFRSCRATCH,$JOBSCRATCH,/lustre/fswork/dataset/"
# rendre les devices/infos IB visibles
#export SINGULARITY_BINDPATH="/dev/infiniband,/sys/class/infiniband,/sys/class/infiniband_verbs,/sys/class/net,/run/udev"

# providers ibverbs + conf (ARM)
#export SINGULARITY_BINDPATH="$SINGULARITY_BINDPATH,/etc/libibverbs.d,/usr/lib,/etc/rdma"

# plugin HPC-X (puisqu’il est chargé dans tes logs)
#export SINGULARITY_BINDPATH="$SINGULARITY_BINDPATH,/opt"

# tes espaces données (tu les avais déjà)
#export SINGULARITY_BINDPATH="$SINGULARITY_BINDPATH,$WORK,$ALL_CCFRWORK,$DSDIR,$SCRATCH,$ALL_CCFRSCRATCH,$JOBSCRATCH,/lustre/fswork/dataset"


## launch script on every task 
set -x
time srun singularity exec --nv --bind $SINGULARITY_BINDPATH \
    $SINGULARITY_ALLOWED_DIR/nemo2506_amd.sif \
    python fsdp2.py --test --model Qwen/Qwen2.5-14B-Instruct --fsdp-checkpointing 3/4 --bsz 2 --grad-acc 1 --compile
date


Overwriting slurm/Sif_fsdp2_h100_14B.slurm


In [89]:
!sbatch slurm/Sif_fsdp2_h100_14B.slurm

sbatch: IDRIS: setting exclusive mode for the job.
Submitted batch job 1685246


In [18]:
!squeue --me

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           1662313    gpu_p5       bc  ssos040 PD       0:00      8 (Priority)
           1662324    gpu_p5       bc  ssos040 PD       0:00      2 (Priority)
           1663702    gpu_p6       bc  ssos040 PD       0:00      8 (Resources)
           1662342    gpu_p6       bc  ssos040  R       4:58      8 jzxh[088,148-149,174,190,322,346-347]
           1662352    gpu_p6       bc  ssos040  R       1:59      8 jzxh[258-262,319-321]


In [25]:
%%writefile slurm/Sif_fsdp2_a100_14B.slurm
#!/bin/bash
#SBATCH --job-name=bc
#SBATCH --output=logs/out/Sif_fsdp2_a100_14B_%j.out 
#SBATCH --error=logs/err/Sif_fsdp2_a100_14B_%j.err
#SBATCH --gres=gpu:8
#SBATCH --nodes=8
#SBATCH --ntasks-per-node=8
#SBATCH --hint=nomultithread 
#SBATCH --time=00:30:00
#SBATCH --cpus-per-task=8
#SBATCH -C a100
#SBATCH --partition=gpu_p5
#SBATCH --account=sos@a100
##SBATCH --qos=qos_gpu_h100-dev 

## load module 
module purge
module load singularity

export NCCL_DEBUG=WARN

## Distribution setup
export MASTER_ADDR=$(scontrol show hostnames $SLURM_JOB_NODELIST | head -n1)
export MASTER_PORT=29500

export SINGULARITY_BINDPATH="/dev/infiniband,/etc/libibverbs.d,/etc/rdma,/sys/class/infiniband,/sys/class/infiniband_verbs,/sys/class/net,/run/udev,$WORK,$ALL_CCFRWORK,$DSDIR,$SCRATCH,$ALL_CCFRSCRATCH,$JOBSCRATCH,/lustre/fswork/dataset/"
# rendre les devices/infos IB visibles
#export SINGULARITY_BINDPATH="/dev/infiniband,/sys/class/infiniband,/sys/class/infiniband_verbs,/sys/class/net,/run/udev"

# providers ibverbs + conf (ARM)
#export SINGULARITY_BINDPATH="$SINGULARITY_BINDPATH,/etc/libibverbs.d,/usr/lib,/etc/rdma"

# plugin HPC-X (puisqu’il est chargé dans tes logs)
#export SINGULARITY_BINDPATH="$SINGULARITY_BINDPATH,/opt"

# tes espaces données (tu les avais déjà)
#export SINGULARITY_BINDPATH="$SINGULARITY_BINDPATH,$WORK,$ALL_CCFRWORK,$DSDIR,$SCRATCH,$ALL_CCFRSCRATCH,$JOBSCRATCH,/lustre/fswork/dataset"


## launch script on every task 
set -x
time srun singularity exec --nv --bind $SINGULARITY_BINDPATH \
    $SINGULARITY_ALLOWED_DIR/nemo2506_amd.sif \
    python fsdp2.py --test --model Qwen/Qwen2.5-14B-Instruct --fsdp-checkpointing 3/4 --bsz 2 --grad-acc 1 --compile
date

Overwriting slurm/Sif_fsdp2_a100_14B.slurm


In [19]:
!sbatch slurm/Sif_fsdp2_a100_14B.slurm

sbatch: IDRIS: setting exclusive mode for the job.
Submitted batch job 1663729


In [97]:
%%writefile slurm/Sif_NeMo_h100_14B.slurm
#!/bin/bash
#SBATCH --job-name=bc
#SBATCH --output=logs/out/Sif_fsdp2_h100_14B_%j.out 
#SBATCH --error=logs/err/Sif_fsdp2_h100_14B_%j.err
#SBATCH --gres=gpu:4
#SBATCH --nodes=8
#SBATCH --ntasks-per-node=4
#SBATCH --hint=nomultithread 
#SBATCH --time=00:30:00
#SBATCH --cpus-per-task=24
#SBATCH -C h100
#SBATCH --partition=gpu_p6
#SBATCH --account=sos@h100
#SBATCH --qos=qos_gpu_h100-dev 

## load module 
module purge
module load singularity

## Distribution setup
export MASTER_ADDR=$(scontrol show hostnames $SLURM_JOB_NODELIST | head -n1)
export MASTER_PORT=29500

export SINGULARITY_BINDPATH="/dev/infiniband,/etc/libibverbs.d,/etc/rdma,$WORK,$ALL_CCFRWORK,$DSDIR,$SCRATCH,$ALL_CCFRSCRATCH,$JOBSCRATCH,/lustre/fswork/dataset/"

## launch script on every task 
set -x
time srun singularity exec --nv --bind $SINGULARITY_BINDPATH \
    $SINGULARITY_ALLOWED_DIR/nemo2506_amd.sif \
    python nemo_test.py
date

Writing slurm/Sif_NeMo_h100_14B.slurm


In [98]:
!sbatch slurm/Sif_NeMo_h100_14B.slurm

sbatch: IDRIS: setting exclusive mode for the job.
Submitted batch job 1577427


In [1]:
!squeue --me

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


In [23]:
!sinfo -s

PARTITION   AVAIL  TIMELIMIT   NODES(A/I/O/T) NODELIST
cpu_p1*        up 4-04:00:00      710/1/9/720 r1i0n[0-35],r1i1n[0-35],r1i2n[0-35],r1i3n[0-35],r1i4n[0-35],r1i5n[0-35],r1i6n[0-35],r1i7n[0-35],r2i0n[0-35],r2i1n[0-35],r2i2n[0-35],r2i3n[0-35],r2i4n[0-35],r2i5n[0-35],r2i6n[0-35],r2i7n[0-35],r3i0n[0-35],r3i1n[0-35],r3i2n[0-35],r3i3n[0-35]
gpu_p13        up 4-04:00:00    291/103/2/396 r3i4n[0-8],r3i5n[0-8],r3i6n[0-8],r3i7n[0-8],r6i0n[0-8],r6i1n[0-8],r6i2n[0-8],r6i3n[0-8],r6i4n[0-8],r6i5n[0-8],r6i6n[0-8],r6i7n[0-8],r7i0n[0-8],r7i1n[0-8],r7i2n[0-8],r7i3n[0-8],r7i4n[0-8],r7i5n[0-8],r7i6n[0-8],r7i7n[0-8],r8i0n[0-8],r8i1n[0-8],r8i2n[0-8],r8i3n[0-8],r8i4n[0-8],r8i5n[0-8],r8i6n[0-8],r8i7n[0-8],r9i0n[0-8],r9i1n[0-8],r9i2n[0-8],r9i3n[0-8],r9i4n[0-8],r9i5n[0-8],r9i6n[0-8],r9i7n[0-8],r10i0n[0-8],r10i1n[0-8],r10i2n[0-8],r10i3n[0-8],r10i4n[0-8],r10i5n[0-8],r10i6n[0-8],r10i7n[0-8]
gpu_v116    inact 4-04:00:00      126/0/0/126 r3i4n[0-8],r3i5n[0-8],r3i6n[0-8],r3i7n[0-8],r6i0n[0-8],r6i1n[0-8],r10i0n[0-

## Nanotron

In [7]:
%%writefile nanotron.slurm
#!/bin/bash
#SBATCH --job-name=bc
#SBATCH --output=logs/out/nanotron_h100_%j.out 
#SBATCH --error=logs/err/nanotron_h100_%j.err
#SBATCH --gres=gpu:4
#SBATCH --nodes=16
#SBATCH --ntasks-per-node=4
#SBATCH --hint=nomultithread 
#SBATCH --time=00:30:00
#SBATCH --cpus-per-task=24
#SBATCH -C h100
#SBATCH --partition=gpu_p6
#SBATCH --account=sos@h100


## load module 
module purge
module load arch/h100
module load pytorch-gpu/py3/2.4.0

export CUDA_DEVICE_MAX_CONNECTIONS=1


## launch script on every task 
set -x
time srun python -u nanotron/run_train.py --config-file config_qwen_TP4_PP16.yaml
date


Overwriting nanotron.slurm


In [8]:
!sbatch nanotron.slurm

sbatch: IDRIS: setting exclusive mode for the job.
Submitted batch job 684681


In [9]:
%pwd

'/lustre/fswork/projects/idris/sos/ssos040/Bench_InstructFT_Tulu3/InstructFT'

In [13]:
!squeue --me

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


In [1]:
#from lightning.pytorch.callbacks.callback import Callback